<a href="https://colab.research.google.com/github/Mihir-Panjikar/Legal_Document_Simplication_System/blob/dev/Notebook/Llama_legal_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade transformers
!pip install bitsandbytes
!pip install --upgrade huggingface_hub
!pip install googletrans

  Using cached transformers-4.50.0-py3-none-any.whl.metadata (39 kB)
Using cached transformers-4.50.0-py3-none-any.whl (10.2 MB)
  Using cached bitsandbytes-0.45.3-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using

In [6]:
from google.colab import userdata
read_token = userdata.get('read_hugging_face')

In [7]:
!huggingface-cli login --token $read_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `Read_access` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Read_access`


In [8]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

/usr/local/lib/python3.11/dist-packages/pynvml.py:6007: RuntimeWarning: coroutine 'Translator.translate' was never awaited
  ('version', c_uint),


In [9]:
model_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True  # Enable CPU offload
)

quantized_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

In [10]:
def format_messages(messages):
    formatted = ""
    for msg in messages:
        formatted += f"{msg['role'].capitalize()}: {msg['content']}\n\n"
    return formatted

In [11]:
input_text = format_messages([
    {"role": "system", "content": """You are an expert in summarization and legal document simplification. Your task is to summarize the following agreement in a way that is easy for a layperson to understand. The summary should include all key details while using simple, clear language and relevant real-life examples where needed. Ensure that no critical information is lost.

Key requirements for the summary:

Use plain English, avoiding complex legal terms.
Provide relevant examples to make the agreement more understandable.
Maintain all important details, such as duration, termination clauses, liability, dispute resolution, and governing law.
Ensure the summary remains concise but does not miss crucial points."""},
    {"role": "user", "content": """This Agreement shall commence on the Effective Date and shall continue for a period of five (5) years unless earlier terminated in accordance with the provisions set forth herein. Either Party may terminate this Agreement upon thirty (30) days prior written notice to the other Party in the event of a material breach that remains uncured for a period of fifteen (15) days following written notice of such breach. In the event of termination, both Parties shall fulfill any obligations incurred prior to the effective date of termination. Furthermore, neither Party shall be liable to the other for any indirect, consequential, or punitive damages arising out of or in connection with this Agreement, except in cases of willful misconduct or gross negligence. Any disputes arising under this Agreement shall be resolved through binding arbitration in accordance with the rules of the American Arbitration Association, and the prevailing Party shall be entitled to recover reasonable attorneys’ fees and costs incurred in connection with such arbitration. This Agreement shall be governed by and construed in accordance with the laws of the State of New York, without regard to its conflict of law principles. If any provision of this Agreement is found to be invalid or unenforceable, the remaining provisions shall continue in full force and effect. No modification, amendment, or waiver of any provision of this Agreement shall be effective unless in writing and signed by both Parties. This Agreement constitutes the entire agreement between the Parties with respect to the subject matter herein and supersedes all prior and contemporaneous agreements, representations, and understandings, whether written or oral."""}
])

In [12]:
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

output = quantized_model.generate(**input_ids, max_new_tokens=1024)

output_text = tokenizer.decode(output[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [14]:
print(output_text)

System: You are an expert in summarization and legal document simplification. Your task is to summarize the following agreement in a way that is easy for a layperson to understand. The summary should include all key details while using simple, clear language and relevant real-life examples where needed. Ensure that no critical information is lost.

Key requirements for the summary:

Use plain English, avoiding complex legal terms.
Provide relevant examples to make the agreement more understandable.
Maintain all important details, such as duration, termination clauses, liability, dispute resolution, and governing law.
Ensure the summary remains concise but does not miss crucial points.

User: This Agreement shall commence on the Effective Date and shall continue for a period of five (5) years unless earlier terminated in accordance with the provisions set forth herein. Either Party may terminate this Agreement upon thirty (30) days prior written notice to the other Party in the event of

In [15]:
from googletrans import Translator

translator = Translator()

translated_text = await translator.translate(output_text, src="en", dest="hi")

print("Translated Text:", translated_text.text)

Translated Text: सिस्टम: आप सारांश और कानूनी दस्तावेज सरलीकरण के विशेषज्ञ हैं। आपका कार्य निम्नलिखित समझौते को इस तरह से संक्षेप में प्रस्तुत करना है जो एक लेपर्सन को समझने के लिए आसान है। सारांश में सरल, स्पष्ट भाषा और प्रासंगिक वास्तविक जीवन के उदाहरणों का उपयोग करते समय सभी प्रमुख विवरण शामिल होने चाहिए जहां आवश्यकता होती है। सुनिश्चित करें कि कोई महत्वपूर्ण जानकारी खो नहीं है।

सारांश के लिए प्रमुख आवश्यकताएं:

जटिल कानूनी शब्दों से बचने के लिए, सादे अंग्रेजी का उपयोग करें।
समझौते को और अधिक समझने योग्य बनाने के लिए प्रासंगिक उदाहरण प्रदान करें।
सभी महत्वपूर्ण विवरणों को बनाए रखें, जैसे कि अवधि, समाप्ति खंड, देयता, विवाद समाधान और शासी कानून।
सुनिश्चित करें कि सारांश संक्षिप्त बना रहे, लेकिन महत्वपूर्ण बिंदुओं को याद नहीं करता है।

उपयोगकर्ता: यह समझौता प्रभावी तिथि पर शुरू होगा और पांच (5) वर्षों की अवधि के लिए जारी रहेगा जब तक कि पहले निर्धारित प्रावधानों के अनुसार समाप्त नहीं किया जाता है। या तो पार्टी इस समझौते को तीस (30) दिनों के पहले लिखित नोटिस पर एक सामग्री उल्लंघन की स्थि